### 練習問題2.69
以下の手続きは、引数として記号・頻度ペアのリ スト(同じ記号が⼆つ以上のペアに出てくることはない)を取り、  
ハフマンアルゴリズムに従ってハフマン符号化木を⽣成する。

    (define (generate-huffman-tree pairs)
        (successive-merge (make-leaf-set pairs)))

make-leaf-setは、上で記述した、ペアのリストを葉の順序つき集合に変換する⼿続きである。  
successive-mergeは、集合の中で重みが最小の要素をmake-code-treeを使って順番にくっつけていき、  
最後に要素がひとつだけ残るようにするというものである。  
その要素が求めるハフマン木となる。  
この手続きを書け  
(この手続きにはちょっと厄介なところがあるが、  
 そこまで複雑ではない。もし⼿続きの設計が複雑になったとしたら、  
 ほぼ確実に何かを間違えている。  
 順序つきの集合表現を使っているということが大きな助けになる)

In [1]:
(define (make-leaf symbol weight) (list 'leaf symbol weight))
(define (leaf? object) (eq? (car object) 'leaf))
(define (symbol-leaf x) (cadr x))
(define (weight-leaf x) (caddr x))

; コンストラクタ
(define (make-code-tree left right)
  (list left right
        (append (symbols left) (symbols right))
        (+ (weight left) (weight right))
    )
  )

; セレクタ
(define (left-branch tree) (car tree))
(define (right-branch tree) (cadr tree))
(define (symbols tree)
  (if (leaf? tree) (list (symbol-leaf tree))
      (caddr tree)
    )
  )
(define (weight tree)
  (if (leaf? tree) (weight-leaf tree)
      (cadddr tree)
    )
  )

(define (decode bits tree)
  (define (decode-1 bits current-branch)
    (if (null? bits) '()
        (let ((next-branch (choose-branch (car bits) current-branch)))
          (if (leaf? next-branch) (cons (symbol-leaf next-branch) (decode-1 (cdr bits) tree))
              (decode-1 (cdr bits) next-branch))
          )
    )
    )
  (decode-1 bits tree)
)  
(define (choose-branch bit branch)
  (cond ((= bit 0) (left-branch branch))
        ((= bit 1) (right-branch branch))
        (else (error "bad bit: CHOOSE-BRANCH" bit))
        )
  )

(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((< (weight x) (weight (car set))) (cons x set))
        (else (cons (car set) (adjoin-set x (cdr set))))))

(define (make-leaf-set pairs)
  (if (null? pairs) '()
      (let ((pair (car pairs)))
        (adjoin-set (make-leaf (car pair) ; symbol
                               (cadr pair)) ; weight
                    (make-leaf-set (cdr pairs)))
        )
    )
  )

In [2]:
; 符号化処理
(define (encode-symbol symbol tree)
  (define (iter sub result)
    (if (leaf? sub)
        (if (eq? (symbol-leaf sub) symbol) result
            '())
        (let ((l (left-branch sub))
              (r (right-branch sub)))
          (let ((l-result (iter l (append result '(0)))))
            (if (not (null? l-result)) l-result
                (iter r (append result '(1)))
              )
            )
        )
      )
    )
  (let ((result (iter tree '())))
    (if (null? result) (error "bad symbol: ENCODE" symbol)
        result
        )
    )
)

; 符号化処理 
(define (encode message tree)
    (if (null? message) '()
        (append
            (encode-symbol (car message) tree)
            (encode (cdr message) tree))
    )
)

In [3]:
; 逆順で返ってくることに注意
(make-leaf-set '((A 4) (B 2) (D 1) (C 1)))

((leaf C 1) (leaf D 1) (leaf B 2) (leaf A 4))

In [4]:
; 間違った実装
(define (successive-merge-ng set)
  (define (iter sub)
    (cond ((null? sub) '())
          ((= (length sub) 1) sub)
          ((= (length sub) 2) (make-code-tree (car sub) (cadr sub)))
          (else (make-code-tree (car sub) (iter (cdr sub))))
        )
    )
  (iter (reverse set))
  )

(define (generate-huffman-tree-ng pairs)
    (successive-merge-ng (make-leaf-set pairs)))

In [5]:
; あまりネストしてない例だとうまく動作しているようにみえる。
(define sample-tree1 (generate-huffman-tree-ng '((A 4) (B 2) (D 1) (C 1))))
sample-tree1

((leaf A 4) ((leaf B 2) ((leaf D 1) (leaf C 1) (D C) 2) (B D C) 4) (A B D C) 8)

In [6]:
(define sample-tree2 (generate-huffman-tree-ng '((A 10) (B 4) (C 1) (D 1) (E 1) (F 1) (G 1) (H 1))))
sample-tree2

((leaf A 10) ((leaf B 4) ((leaf C 1) ((leaf D 1) ((leaf E 1) ((leaf F 1) ((leaf G 1) (leaf H 1) (G H) 2) (F G H) 3) (E F G H) 4) (D E F G H) 5) (C D E F G H) 6) (B C D E F G H) 10) (A B C D E F G H) 20)

In [7]:
(display (encode '(A) sample-tree1))
(newline)
(display (encode '(B) sample-tree1))
(newline)
(display (encode '(C) sample-tree1))
(newline)
(display (encode '(D) sample-tree1))
(newline)


(0)
(1 0)
(1 1 1)
(1 1 0)


In [8]:
(display (encode '(A) sample-tree2))
(newline)
(display (encode '(B) sample-tree2))
(newline)
(display (encode '(C) sample-tree2))
(newline)
(display (encode '(D) sample-tree2))
(newline)
(display (encode '(E) sample-tree2))
(newline)
(display (encode '(F) sample-tree2))
(newline)
(display (encode '(G) sample-tree2))
(newline)
(display (encode '(H) sample-tree2))
(newline)

(0)
(1 0)
(1 1 0)
(1 1 1 0)
(1 1 1 1 0)
(1 1 1 1 1 0)
(1 1 1 1 1 1 0)
(1 1 1 1 1 1 1)


In [9]:
; 考え直した回答

; 指定した頻度がリストに含まれているか。
; 含まれていない場合、falseを返す。
; 含まれている場合、最初に見つかったペアと次のペアを返す。（これの2つで木を生成する）
(define (have-weight? w set)
  ;(display "have-weight? ")
  ;(display w)
  ;(display " ")
  ;(display set)
  ;(display " ")
  ;(newline)
  (cond ((null? set) #f)
        ((not (pair? set))
         (if (= (weight set) w) (list set)
             #f
             )
         )
        (else
         (if (= (weight (car set)) w) (list (car set) (cadr set))
             (have-weight? w (cdr set))
             )
         )
    )
)

; ハフマン符号木の生成の記号・頻度ペアのリストから、指定したペアを削除したリストを返す。
(define (remove-info item set)
  (if (null? set) '()
      (if (equal? item (car set)) (remove-info item (cdr set))
          (cons (car set) (remove-info item (cdr set)))
      )
  )
  )

; 回答
(define (successive-merge set)
  (define (iter w result)
    ;(display result)
    ;(newline)
    (cond ((null? result) '())
          ((= (length result) 1) (car result))
          ;((not (pair? result)) result) ; これは動作しない
          ((= (length result) 2) (make-code-tree (cadr result) (car result)))
          (else
           (let ((ll (have-weight? w result)))
             (if (equal? ll #f) (iter (+ w 1) result)
                 (let ((top (car ll))
                       (next (cadr ll)))
                   (let ((new-result (remove-info next (remove-info top result))))
                     (if (null? new-result) (iter w (list (make-code-tree next top)))
                         (if (= (length new-result) 1) (iter w (append (list (make-code-tree next top)) new-result))
                             (let (;(new-item (make-code-tree next top))
                                   (new-item (if (>= (weight next) (weight top)) (make-code-tree next top) (make-code-tree top next)))
                                   )
                               (if (<= (weight new-item) (weight (car new-result)))
                                   (iter w (append (list new-item) new-result))
                                   (iter w (append new-result (list new-item)))
                                )
                              )
                           )
                        )
                     )
                    )
                )
             )
           )
        )
    )
  (iter 1 set)
  )

(define (generate-huffman-tree pairs)
    (successive-merge (make-leaf-set pairs)))

In [10]:
(generate-huffman-tree '((A 4) (B 2) (D 1) (C 1)))

((leaf A 4) ((leaf B 2) ((leaf D 1) (leaf C 1) (D C) 2) (B D C) 4) (A B D C) 8)

In [11]:
; テキストのハフマン符号木の生成の記号・頻度ペアのリスト
; これだとうまく動作しない
; ->修正してできるようになった
(define sample-tree3 (generate-huffman-tree '((A 8) (B 3) (C 1) (D 1) (E 1) (F 1) (G 1) (H 1))))
sample-tree3

((leaf A 8) (((leaf B 3) ((leaf C 1) (leaf D 1) (C D) 2) (B C D) 5) (((leaf E 1) (leaf F 1) (E F) 2) ((leaf G 1) (leaf H 1) (G H) 2) (E F G H) 4) (B C D E F G H) 9) (A B C D E F G H) 17)

In [12]:
; 左≧右
(define sample-tree4 (generate-huffman-tree '((A 8) (B 2) (C 1) (D 1) (E 1) (F 1) (G 1) (H 1))))
sample-tree4

((leaf A 8) (((leaf B 2) ((leaf C 1) (leaf D 1) (C D) 2) (B C D) 4) (((leaf E 1) (leaf F 1) (E F) 2) ((leaf G 1) (leaf H 1) (G H) 2) (E F G H) 4) (B C D E F G H) 8) (A B C D E F G H) 16)

In [13]:
; 単一の記号で動作確認
(display (encode '(A) sample-tree3))
(newline)
(display (encode '(B) sample-tree3))
(newline)
(display (encode '(C) sample-tree3))
(newline)
(display (encode '(D) sample-tree3))
(newline)
(display (encode '(E) sample-tree3))
(newline)
(display (encode '(F) sample-tree3))
(newline)
(display (encode '(G) sample-tree3))
(newline)
(display (encode '(H) sample-tree3))
(newline)

(0)
(1 0 0)
(1 0 1 0)
(1 0 1 1)
(1 1 0 0)
(1 1 0 1)
(1 1 1 0)
(1 1 1 1)


In [14]:
(encode '(B A C A D A E A F A B B A A A G A H) sample-tree3)

(1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1)

In [15]:
(decode '(1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1) sample-tree3)

(B A C A D A E A F A B B A A A G A H)